In [2]:
from serpapi import GoogleSearch
from dotenv import load_dotenv
import os

load_dotenv()

params = {
    "engine": "google_events",
    "q": "Events in Durham",
    "hl": "en",
    "gl": "us",
    "api_key": os.environ["SEARCH_API_KEY"],
}

search = GoogleSearch(params)
results = search.get_dict()
events_results = results["events_results"]
print(events_results)

[{'title': 'Winter Food Truck Rodeo!', 'date': {'start_date': 'Feb 23', 'when': 'Sun, Feb 23, 12 – 4 PM EST'}, 'address': ['Durham Central Park, 501 Foster St', 'Durham, NC'], 'link': 'https://www.glass-jug.com/events/foodtruckrodeofeb25', 'event_location_map': {'image': 'https://www.google.com/maps/vt/data=S2F0GwuQT89Cc282wKn7dxlALdnonFMEKMu5bGWUbaNuv8nNdkUggbbwu8peLZBioIp4PHBAkHiNllib-r9-F3s7-snd1HfSjNv52pLNWSr25_STppY', 'link': 'https://www.google.com/maps/place//data=!4m2!3m1!1s0x89ace473c2a1928d:0x2b3f3f4dc8b7cf39?sa=X&ved=2ahUKEwj6-d-rhtiLAxVhg4kEHdV8LPMQ9eIBegQIARAA&hl=en&gl=us', 'serpapi_link': 'https://serpapi.com/search.json?data=%214m2%213m1%211s0x89ace473c2a1928d%3A0x2b3f3f4dc8b7cf39&engine=google_maps&gl=us&google_domain=google.com&hl=en&q=Events+in+Durham&type=place'}, 'description': 'Join us downtown for over 50+ food trucks, cuisine from all over the world, and family fun - making it the ultimate Sunday Funday in Durham Central Park! The famous rodeos feature trucks and

In [ ]:
from serpapi import GoogleSearch
from dotenv import load_dotenv
import os
import json

# Load environment variables
load_dotenv()

# Define search parameters
params = {
    "engine": "google_events",
    "q": "Events in Wilmington, NC",
    "hl": "en",
    "gl": "us",
    "api_key": os.environ["SEARCH_API_KEY"],
}

# Perform search
search = GoogleSearch(params)
results = search.get_dict()

# Extract event results
events_results = results.get("events_results", [])

# Save results to a JSON file
output_file = "json_output/events_results.json"
with open(output_file, "w", encoding="utf-8") as json_file:
    json.dump(events_results, json_file, indent=4, ensure_ascii=False)

print(f"Results saved to {output_file}")

Results saved to events_results.json


In [1]:
from serpapi import GoogleSearch
from dotenv import load_dotenv
import os
import json
import geocoder

load_dotenv()

# Fetch user's location automatically based on IP
g = geocoder.ip("me")
user_location = (
    g.city + ", " + g.state if g.city and g.state else "USA"
)  # Fallback to "USA" if location fails

print(f"Detected location: {user_location}")

# Define search parameters with auto-detected location
params = {
    "engine": "google_events",
    "q": f"Events in {user_location}",
    "hl": "en",
    "gl": "us",
    "api_key": os.environ["SEARCH_API_KEY"],
}

# Perform search
search = GoogleSearch(params)
results = search.get_dict()

# Extract event results
events_results = results.get("events_results", [])

# Save results to a JSON file
output_file = "json_output/events_results.json"
with open(output_file, "w", encoding="utf-8") as json_file:
    json.dump(events_results, json_file, indent=4, ensure_ascii=False)

print(f"Results saved to {output_file}")

Detected location: Durham, North Carolina
Results saved to json_output/events_results.json


In [ ]:
## THIS HAS BEEN MOVED TO geministuff.py

import sqlite3
import os
import shutil
import pandas as pd

from datetime import datetime, timedelta


# Detect OS and set the correct Chrome history path

if os.name == "nt":  # Windows

    history_db_path = os.path.expanduser(
        r"~\AppData\Local\Google\Chrome\User Data\Default\History"
    )

elif os.name == "posix":  # Mac/Linux

    history_db_path = os.path.expanduser(
        "~/Library/Application Support/Google/Chrome/Default/History"
    )  # Adjust for Linux if needed


# Create a temporary copy of the history database to avoid lock issues

temp_db_path = "./chrome_history_temp.db"

shutil.copy2(history_db_path, temp_db_path)


# Connect to the copied database

conn = sqlite3.connect(temp_db_path)

cursor = conn.cursor()


# Query to fetch browsing history

query = """

SELECT url, title, visit_count, last_visit_time 

FROM urls

ORDER BY last_visit_time DESC

LIMIT 100;  -- Adjust the limit as needed
"""


cursor.execute(query)

history_data = cursor.fetchall()


# Convert Chrome's timestamp format to human-readable datetime


def convert_chrome_timestamp(timestamp):
    """Convert Chrome's timestamp format to human-readable datetime."""

    return datetime(1601, 1, 1) + timedelta(microseconds=timestamp)


# Store history in a Pandas DataFrame

df = pd.DataFrame(
    history_data, columns=["URL", "Title", "Visit Count", "Last Visit Time"]
)

df["Last Visit Time"] = df["Last Visit Time"].apply(convert_chrome_timestamp)


# Close connection

conn.close()


# Display results

from IPython.display import display


display(df)


# Save history to JSON (optional)

df.to_json("chrome_browsing_history.json", orient="records", indent=4)

In [2]:
import json
import time
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

# Load JSON file
file_path = "json_output/events_results.json"
with open(file_path, "r", encoding="utf-8") as f:
    events_data = json.load(f)

# Initialize Geopy geocoder
geolocator = Nominatim(user_agent="geocoding_app", timeout=10)

def get_lat_long(address, retries=3):
    """
    Converts an address to latitude and longitude using Geopy.
    Retries up to 3 times if it times out.
    
    Args:
        address (str): The address to geocode.
        retries (int): Number of retry attempts if Geopy fails.
    
    Returns:
        tuple: (latitude, longitude) or (None, None) if not found.
    """
    for attempt in range(retries):
        try:
            location = geolocator.geocode(address)
            if location:
                return location.latitude, location.longitude
        except GeocoderTimedOut:
            print(f"Geocoder timed out for '{address}', retrying ({attempt+1}/{retries})...")
        time.sleep(2)  # Delay between retries to avoid rate limits
    return None, None

# Process each event and get coordinates
for event in events_data:
    original_address = ", ".join(event["address"])  # Full address
    street_address = event["address"][0]  # Extract only street name
    city_state = event["address"][-1]  # Extract city and state

    # Try different formats for better results
    possible_addresses = [
        original_address,  # Full address
        street_address + ", " + city_state,  # Street + City/State
        city_state  # City and state only
    ]

    lat, lon = None, None
    for addr in possible_addresses:
        lat, lon = get_lat_long(addr)
        if lat and lon:
            break  # Stop once a valid lat/lon is found

    # Save latitude & longitude
    event["latitude"] = lat
    event["longitude"] = lon

    print(f"Processed: {event['title']} -> ({lat}, {lon})")

    time.sleep(2)  # Prevent hitting API rate limits

# Save updated JSON with coordinates
output_file = "json_output/events_results.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(events_data, f, indent=4, ensure_ascii=False)

print(f"Updated file saved to: {output_file}")


Processed: Winter Food Truck Rodeo! -> (35.996653, -78.9018053)
Processed: Trae Crowder -> (35.996653, -78.9018053)
Processed: Dinner Detective Raleigh – Durham, NC - February 22, 2025 @ 6:00 PM -> (35.84476885, -78.67925077388523)
Processed: Open Mic Event -> (35.7803977, -78.6390989)
Processed: 🎸 Steve Martin -> (35.996653, -78.9018053)
Processed: The Dinner Detective Comedy Mystery Dinner Show - Raleigh-Durham, NC -> (35.84476885, -78.67925077388523)
Processed: Private Event -> (35.996653, -78.9018053)
Processed: Dancing with the Stars -> (35.996653, -78.9018053)
Processed: Open Mic Event -> (35.9144636, -78.78281166902025)
Processed: FAMILY CONCERT: Everything you wanted to know about the LUTE…… -> (35.994904700000006, -78.89665448525244)
Updated file saved to: json_output/events_results.json
